In [1]:
# VERSION 1
from google.colab import files
files.upload()
import pandas as pd
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', None)
df = pd.read_excel('Online Retail.xlsx')

# Data Checking
## Column Type
print('Column Type Check')
print(df.info())
## Blank Check
print('\nBlank Check')
print((df.isna().mean()*100)[(df.isna().mean()*100) > 0].sort_values(ascending= False))
## Outliers Check
print('\nOutliers Check')
report = {}
outliers_rows = pd.DataFrame()
for col in df.select_dtypes(include= 'number').columns:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)
  iqr = q3 -q1
  lower_bound = q1 - iqr * 1.5
  upper_bound = q3 + iqr * 1.5
  outliers = ((df[col] < lower_bound) | (df[col] > upper_bound))
  outliers_pct = outliers.mean()*100
  if outliers_pct > 0:
    report[col] = round(outliers_pct, 2)
    outliers_rows = pd.concat([outliers_rows, df[outliers]])
outliers_report = pd.Series(report, name = 'outlier_pct').sort_values(ascending= False)
print(outliers_rows.head(10))
print(outliers_report)
## Duplicate Check
print('\nDuplicate Check')
print(df.duplicated().sum())
print(df[df.duplicated(keep= False)].head(10))

# Data Cleaning
df = df.dropna(subset= ['CustomerID'])
df['Description'] = df['Description'].fillna('Unknown')
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
df = df.drop_duplicates()
df['CustomerID'] = df['CustomerID'].astype(int).astype(str)

# Export Cleaned Data
df.to_csv('Online Retail_cleaned.csv', index= False)
files.download('Online Retail_cleaned.csv')

Saving Online Retail.xlsx to Online Retail.xlsx
Column Type Check
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None

Blank Check
CustomerID     24.926694
Description     0.268311
dtype: float64

Outliers Check
   InvoiceNo StockCode                          Description  Quantity  \
9     536367     84879        ASSORTED COLOUR BIRD ORNAMENT        32 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [2]:
from google.colab import files
files.upload()
import pandas as pd
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', None)
df_cleaned = pd.read_csv('Online Retail_cleaned.csv', low_memory= False)
basket = df_cleaned.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0)
basket_set = basket.map(lambda x: 1 if x > 0 else 0)
from mlxtend.frequent_patterns import fpgrowth, association_rules
frequent_items = fpgrowth(basket_set, min_support= 0.01, use_colnames= True)
rules = association_rules(frequent_items, metric= 'lift', min_threshold= 1.01)
print('Top 10 by Support')
print(rules.sort_values('support', ascending= False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))
print('\nTop 10 by Confidence')
print(rules.sort_values('confidence', ascending= False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))
print('\nTop 10 by Lift')
print(rules.sort_values('lift', ascending= False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

Saving Online Retail_cleaned.csv to Online Retail_cleaned (1).csv


/usr/local/lib/python3.12/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: Dep

Top 10 by Support
                            antecedents                         consequents  \
63            (JUMBO BAG RED RETROSPOT)           (JUMBO BAG PINK POLKADOT)   
62            (JUMBO BAG PINK POLKADOT)           (JUMBO BAG RED RETROSPOT)   
520   (GREEN REGENCY TEACUP AND SAUCER)  (ROSES REGENCY TEACUP AND SAUCER )   
521  (ROSES REGENCY TEACUP AND SAUCER )   (GREEN REGENCY TEACUP AND SAUCER)   
19         (ALARM CLOCK BAKELIKE GREEN)         (ALARM CLOCK BAKELIKE RED )   
18          (ALARM CLOCK BAKELIKE RED )        (ALARM CLOCK BAKELIKE GREEN)   
261           (LUNCH BAG RED RETROSPOT)           (LUNCH BAG PINK POLKADOT)   
260           (LUNCH BAG PINK POLKADOT)           (LUNCH BAG RED RETROSPOT)   
425           (LUNCH BAG RED RETROSPOT)           (LUNCH BAG  BLACK SKULL.)   
424           (LUNCH BAG  BLACK SKULL.)           (LUNCH BAG RED RETROSPOT)   

      support  confidence       lift  
63   0.029463    0.341250   7.260672  
62   0.029463    0.626866   7.2606

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag